**For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.**

1. Start by creating a new Notebook for this assignment.

2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:


3. To create the above dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

4. Submit a link to your Notebook on your Github repository. (10 marks)

# **Part 1**

**Tried multiple times with multiple scraping methods, however, finally had to use an older version of Wiki page to get the right Table**

Importing all necessary libraries

In [0]:
import requests #with this we can add content like headers, form data, multipart files, and parameters 
import lxml.html as lh #for processing html content
import bs4 as bs #web scraping
import numpy as np 
import pandas as pd #for dfs

In [0]:
#latest versions were giving a lot of errors, so I used an older version of the Wiki page

url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=942655364"
res = requests.get(url)
soup = bs.BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
data = pd.read_json(df[0].to_json(orient='records'))

**Q - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood**

**Observation - The table that I got has POSTCODE instead of PostalCode due to the older version.**

In [8]:
data.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
data

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


**If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.**

In [0]:
#reordering and grouping data

data_bor = data[data['Borough'] != 'Not assigned']
data_bor = data_bor.groupby(['Borough', 'Postcode'], as_index=False).agg(','.join)

In [0]:
#replacing values with Non Assigned Neigbourhoods
data_bor['Neighbourhood'] = np.where(data_bor['Neighbourhood'] == 'Not assigned', data_bor['Borough'], data_bor['Neighbourhood'])

In [12]:
data_bor

,Borough,Postcode,Neighbourhood
0,Central Toronto,M4N,Lawrence Park
1,Central Toronto,M4P,Davisville North
2,Central Toronto,M4R,North Toronto West
3,Central Toronto,M4S,Davisville
4,Central Toronto,M4T,"Moore Park,Summerhill East"
...,...,...,...
98,York,M6C,Humewood-Cedarvale
99,York,M6E,Caledonia-Fairbanks
100,York,M6M,"Del Ray,Keelesdale,Mount Dennis,Silverthorn"
101,York,M6N,"The Junction North,Runnymede"


In [13]:
data_bor.shape

(103, 3)

**Our Data contains 103 Rows and 3 Columns**
End of Part 1
---



# **Part 2**

Importing all Necessary libraries just to be on the safe side



In [0]:
import requests
import lxml.html as lh
import bs4 as bs
import urllib.request
import numpy as np 
import pandas as pd

**I uploaded the geospatial coordinates file first in my virtual colab drive and then used its location below.**

In [2]:
geo_data = "/content/Geospatial_Coordinates.csv"
geospatial_data = pd.read_csv(geo_data)
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**As Postal Code was renamed as Postcode, we will change it here as well**

In [28]:

geospatial_data.columns = ['Postcode', 'Latitude', 'Longitude']
geospatial_data.columns

Index(['Postcode', 'Latitude', 'Longitude'], dtype='object')

In [38]:
geospatial_data

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


**Now we will merge both the tables to create a new dataframe "Geo_Tbl"**

In [34]:
geo_tbl = data_bor.merge(geospatial_data, left_on="Postcode", right_on="Postcode")
geo_tbl.head()

,Borough,Postcode,Neighbourhood,Latitude,Longitude
0,Central Toronto,M4N,Lawrence Park,43.728020,-79.388790
1,Central Toronto,M4P,Davisville North,43.712751,-79.390197
2,Central Toronto,M4R,North Toronto West,43.715383,-79.405678
3,Central Toronto,M4S,Davisville,43.704324,-79.388790
4,Central Toronto,M4T,"Moore Park,Summerhill East",43.689574,-79.383160


**Now we will reorder to Columns using Pandas simple code to reorder the columns as per requirement**

In [36]:
geo_tbl = geo_tbl[['Postcode','Borough','Neighbourhood','Latitude','Longitude']]
geo_tbl

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
...,...,...,...,...,...
98,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
99,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
100,M6M,York,"Del Ray,Keelesdale,Mount Dennis,Silverthorn",43.691116,-79.476013
101,M6N,York,"The Junction North,Runnymede",43.673185,-79.487262


End of Part 2

---



In [0]:
import pandas as pd

# **Part 3**

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.

**Importing all libraries from the New York lab notebook**

In [47]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [0]:
from pandas.io.json import json_normalize 
import json 

We will also first import Geopy and heck the geo cordinates of Toronto in order to create a specific map area for Toronto

In [44]:
!pip install geopy
import geopy

In [46]:
address = 'Toronto, Ontario'

geolocator = geopy.Nominatim(user_agent="ny_explorer", timeout=30)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Toronto Cord. {}, {}.'.format(latitude, longitude))

Toronto Cord. 43.6534817, -79.3839347.


We will now create a map using cooridnates from our table Geo_Tbl

In [63]:

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, neighbourhood in zip(geo_tbl['Latitude'], geo_tbl['Longitude'], geo_tbl['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [0]:
#Defining our credentials for fourquare

CLIENT_ID = #removed this for peer review
CLIENT_SECRET = #removed this for peer review
VERSION = '20161225' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

We will limit our search to 8 venues each in a radius of 400m

In [0]:
LIMIT = 8
radius = 400

location_list = [] 

for neighbourhood, latitude, longitude in zip(geo_tbl.Neighbourhood, geo_tbl.Latitude, geo_tbl.Longitude):
    
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
    
    data = requests.get(url).json()
    
    # use len() to check if any data within "items" (len = 0 if nothing)
    length = len(data['response']['groups'][0]['items'])
    if length == 0:
        continue 
    else:
        venue = data['response']['groups'][0]['items'][0]['venue']
        
        # extract info within 'venue'
        name = venue['name']
        lat = venue['location']['lat']
        lon = venue['location']['lng']
        cat = venue['categories'][0]['name']
        
        location_list.append([(neighbourhood, latitude, longitude, name, lat, lon, cat)])

We will create a new DF with desired columns

In [69]:
foursqdf = pd.DataFrame(x for row in location_list for x in row)
foursqdf.columns = ['Neighbourhood','Latitude','Longitude','Venue','Latitude','Longitude','category']
foursqdf.head()

,Neighbourhood,Latitude,Longitude,Venue,Latitude,Longitude,category
0,Davisville North,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop
1,North Toronto West,43.715383,-79.405678,St. Clements - Yonge Parkette,43.712062,-79.404255,Park
2,Davisville,43.704324,-79.388790,Jules Cafe Patisserie,43.704138,-79.388413,Dessert Shop
3,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,The Market By Longo’s,43.686711,-79.399536,Supermarket
4,"Forest Hill North,Forest Hill West",43.696948,-79.411307,Forest Hill Road Park,43.697945,-79.406605,Park


In [78]:
#Number of nearby locations for our neighbourhoods
format(len(foursqdf.Venue))

'92'

# **Clustering and Analyzing Each Neighborhood using get_dummies function from Lab**

In [82]:
dummydata = pd.get_dummies(foursqdf.category) 
df_01 = pd.concat([foursqdf[['Neighbourhood']], dummydata], axis=1)
df_01.head()

,Neighbourhood,Airport,Arts & Crafts Store,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,Building,Cafeteria,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Field,Fish & Chips Shop,Food,Food & Drink Shop,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Hakka Restaurant,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Liquor Store,Massage Studio,Motel,Neighborhood,Park,Pizza Place,Playground,Pool,Rental Car Location,Sandwich Place,Skating Rink,Sports Bar,Supermarket,Thai Restaurant,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Warehouse Store,Wings Joint
0,Davisville North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,North Toronto West,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Davisville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,"Forest Hill North,Forest Hill West",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Importing K Means and Running various clusters

In [0]:
from sklearn.cluster import KMeans

In [100]:
final_df = df_01.drop('Neighbourhood', axis=1)

n_group = 5 


kmeans = KMeans(n_clusters=n_group, random_state=0).fit(final_df)


kmeans.labels_[0:10]

array([0, 1, 0, 0, 1, 4, 0, 0, 0, 0], dtype=int32)

In [95]:
df_01.head()

,Neighbourhood,label,Airport,Arts & Crafts Store,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,Building,Cafeteria,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Field,Fish & Chips Shop,Food,Food & Drink Shop,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Hakka Restaurant,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Liquor Store,Massage Studio,Motel,Neighborhood,Park,Pizza Place,Playground,Pool,Rental Car Location,Sandwich Place,Skating Rink,Sports Bar,Supermarket,Thai Restaurant,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Warehouse Store,Wings Joint
0,Davisville North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,North Toronto West,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Davisville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,"Forest Hill North,Forest Hill West",2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Merging this DF with our previously saved GEO_TBL

In [102]:
final_merged = pd.merge(geo_tbl, df_01, on='Neighbourhood', how='right')
final_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,label,Airport,Arts & Crafts Store,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,Building,Cafeteria,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Field,Fish & Chips Shop,Food,Food & Drink Shop,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Hakka Restaurant,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Liquor Store,Massage Studio,Motel,Neighborhood,Park,Pizza Place,Playground,Pool,Rental Car Location,Sandwich Place,Skating Rink,Sports Bar,Supermarket,Thai Restaurant,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Warehouse Store,Wings Joint
0,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Using the code in our lab exercise, we will now create a Map showing various locations and clusters on the map

In [112]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)


x = np.arange(n_group)
ys = [i + x + (i*x)**2 for i in range(n_group)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(final_merged['Latitude'], final_merged['Longitude'], final_merged['Neighborhood'], final_merged['label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now that we see them on the map, we will run the size of all 5 clusters

In [106]:
final_merged.loc[final_merged['label'] == 0, final_merged.columns[[1] + list(range(5, final_merged.shape[1]))]]

,Borough,label,Airport,Arts & Crafts Store,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,Building,Cafeteria,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Field,Fish & Chips Shop,Food,Food & Drink Shop,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Hakka Restaurant,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Liquor Store,Massage Studio,Motel,Neighborhood,Park,Pizza Place,Playground,Pool,Rental Car Location,Sandwich Place,Skating Rink,Sports Bar,Supermarket,Thai Restaurant,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Warehouse Store,Wings Joint
0,Central Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Central Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Central Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
6,Downtown Toronto,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Downtown Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Downtown Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
9,Downtown Toronto,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,Downtown Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11,Downtown Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,Downtown Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [107]:
final_merged.loc[final_merged['label'] == 1, final_merged.columns[[1] + list(range(5, final_merged.shape[1]))]]

,Borough,label,Airport,Arts & Crafts Store,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,Building,Cafeteria,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Field,Fish & Chips Shop,Food,Food & Drink Shop,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Hakka Restaurant,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Liquor Store,Massage Studio,Motel,Neighborhood,Park,Pizza Place,Playground,Pool,Rental Car Location,Sandwich Place,Skating Rink,Sports Bar,Supermarket,Thai Restaurant,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Warehouse Store,Wings Joint
16,Downtown Toronto,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,Downtown Toronto,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22,Downtown Toronto,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [108]:
final_merged.loc[final_merged['label'] == 2, final_merged.columns[[1] + list(range(5, final_merged.shape[1]))]]

,Borough,label,Airport,Arts & Crafts Store,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,Building,Cafeteria,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Field,Fish & Chips Shop,Food,Food & Drink Shop,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Hakka Restaurant,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Liquor Store,Massage Studio,Motel,Neighborhood,Park,Pizza Place,Playground,Pool,Rental Car Location,Sandwich Place,Skating Rink,Sports Bar,Supermarket,Thai Restaurant,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Warehouse Store,Wings Joint
1,Central Toronto,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Central Toronto,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24,Downtown Toronto,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
34,East York,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
37,Etobicoke,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
43,Etobicoke,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
54,North York,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59,North York,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60,North York,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
87,West Toronto,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [109]:
final_merged.loc[final_merged['label'] == 3, final_merged.columns[[1] + list(range(5, final_merged.shape[1]))]]

,Borough,label,Airport,Arts & Crafts Store,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,Building,Cafeteria,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Field,Fish & Chips Shop,Food,Food & Drink Shop,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Hakka Restaurant,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Liquor Store,Massage Studio,Motel,Neighborhood,Park,Pizza Place,Playground,Pool,Rental Car Location,Sandwich Place,Skating Rink,Sports Bar,Supermarket,Thai Restaurant,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Warehouse Store,Wings Joint
36,Etobicoke,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
53,North York,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
68,North York,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [110]:
final_merged.loc[final_merged['label'] == 4, final_merged.columns[[1] + list(range(5, final_merged.shape[1]))]]

,Borough,label,Airport,Arts & Crafts Store,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,Building,Cafeteria,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Concert Hall,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Field,Fish & Chips Shop,Food,Food & Drink Shop,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Hakka Restaurant,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Liquor Store,Massage Studio,Motel,Neighborhood,Park,Pizza Place,Playground,Pool,Rental Car Location,Sandwich Place,Skating Rink,Sports Bar,Supermarket,Thai Restaurant,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Warehouse Store,Wings Joint
5,Central Toronto,4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
64,North York,4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
79,Scarborough,4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
